In [1]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

In [2]:
model = ChatOpenAI(model="gpt-4o")

In [3]:
from tools import add_employee_learning_status, create_one_mail_draft

In [5]:
employee_agent = create_react_agent(
    model=model,
    tools=[add_employee_learning_status],
    name="employee_agent",
    prompt="You are an employee agent responsible for managing employee learning status records. You have access to tools that can add employee learning status information to the system. Always use one tool at a time and only when necessary."
)

mail_agent = create_react_agent(
    model=model,
    tools=[create_one_mail_draft],
    name="mail_agent",
    prompt="You are a mail agent responsible for creating mail drafts. You have access to tools that can create mail drafts. Always use one tool at a time and only when necessary. You can't create more than one mail draft at a time."
)

# Create supervisor workflow
workflow = create_supervisor(
    [employee_agent, mail_agent],
    model=model,
    output_mode="last_message", # "full_history"
    prompt=(
        "You are a team supervisor managing a employee agent and a mail agent. "
        "For employee learning status records, use employee_agent. "
        "For mail drafts, use mail_agent."
    )
)

In [6]:
# Compile and run
app = workflow.compile() # TODO add memory

In [7]:
for chunk in app.stream({
    "messages": [
        {
            "role": "user",
            "content": "Can you please remind lucas@mail.com that he has to attend the training on 28/02/2025?"
        }
    ]
}, stream_mode="updates"):
    print(chunk)

{'supervisor': [{'messages': [HumanMessage(content='Can you please remind lucas@mail.com that he has to attend the training on 28/02/2025?', additional_kwargs={}, response_metadata={}, id='d14852e3-6d08-40f5-b342-fec0f18321f5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dOnxYZu5Cg3nqPG7q7ePUqBe', 'function': {'arguments': '{}', 'name': 'transfer_to_mail_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 111, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-a208fce7-20c2-4dfd-bc67-5a1e2c1e5b29-0', tool_calls=[{'name': 'transfer_to_mail_agent', 'args': {}, 'id': 'c

In [9]:
# Invoke with initial state
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Add learning status for employee John Doe: completed Python course on 2023-05-15 with score 95/100"
        }
    ]
})

# Display the result
print("Final result:")
print(result)


Final result:
{'messages': [HumanMessage(content='Add learning status for employee John Doe: completed Python course on 2023-05-15 with score 95/100', additional_kwargs={}, response_metadata={}, id='f7974475-b0d9-4dc8-9bb6-76c3b97b3e6e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_irDo0wrxiF93HMMp6EKvY4Hk', 'function': {'arguments': '{}', 'name': 'transfer_to_employee_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 112, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-72dbee78-3cad-4c06-a8d7-52068f4f3e34-0', tool_calls=[{'name': 'transfer_to_employee_agent', 'a

In [10]:
result

{'messages': [HumanMessage(content='Add learning status for employee John Doe: completed Python course on 2023-05-15 with score 95/100', additional_kwargs={}, response_metadata={}, id='f7974475-b0d9-4dc8-9bb6-76c3b97b3e6e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_irDo0wrxiF93HMMp6EKvY4Hk', 'function': {'arguments': '{}', 'name': 'transfer_to_employee_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 112, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-72dbee78-3cad-4c06-a8d7-52068f4f3e34-0', tool_calls=[{'name': 'transfer_to_employee_agent', 'args': {}, 'i